# Data Cleaning Utility  
Syntecxhub Data Science Internship – Week 1  

## Objective
Build a reusable data cleaning pipeline that:
- Standardizes column names
- Converts date columns
- Removes duplicates
- Handles missing values automatically
- Generates a cleaning report
- Exports cleaned dataset

In [15]:
import os
os.listdir("data")

['raw_data.csv']

In [16]:
import pandas as pd
import numpy as np

print("Loading dataset...")

df = pd.read_csv("data/raw_data.csv")

print("Dataset loaded successfully!")
print("Shape:", df.shape)

df.head()

Loading dataset...
Dataset loaded successfully!
Shape: (9800, 18)


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [17]:
print("Shape of dataset:", df.shape)
print("\nColumn Names:")
print(df.columns)

print("\nData Types:")
print(df.dtypes)

Shape of dataset: (9800, 18)

Column Names:
Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales'],
      dtype='str')

Data Types:
Row ID             int64
Order ID             str
Order Date           str
Ship Date            str
Ship Mode            str
Customer ID          str
Customer Name        str
Segment              str
Country              str
City                 str
State                str
Postal Code      float64
Region               str
Product ID           str
Category             str
Sub-Category         str
Product Name         str
Sales            float64
dtype: object


In [18]:
print("\nMissing Values:")
print(df.isnull().sum())


Missing Values:
Row ID            0
Order ID          0
Order Date        0
Ship Date         0
Ship Mode         0
Customer ID       0
Customer Name     0
Segment           0
Country           0
City              0
State             0
Postal Code      11
Region            0
Product ID        0
Category          0
Sub-Category      0
Product Name      0
Sales             0
dtype: int64


In [19]:
# Standardize column names
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

print("Updated Columns:")
print(df.columns)

Updated Columns:
Index(['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode',
       'customer_id', 'customer_name', 'segment', 'country', 'city', 'state',
       'postal_code', 'region', 'product_id', 'category', 'sub-category',
       'product_name', 'sales'],
      dtype='str')


In [20]:
# Check duplicate rows
duplicates = df.duplicated().sum()
print("Number of duplicate rows:", duplicates)

Number of duplicate rows: 0


In [21]:
# Convert date columns to datetime
df["order_date"] = pd.to_datetime(df["order_date"], errors="coerce")
df["ship_date"] = pd.to_datetime(df["ship_date"], errors="coerce")

print(df[["order_date", "ship_date"]].dtypes)

order_date    datetime64[us]
ship_date     datetime64[us]
dtype: object


In [22]:
df.isnull().sum()

row_id              0
order_id            0
order_date       5841
ship_date        5985
ship_mode           0
customer_id         0
customer_name       0
segment             0
country             0
city                0
state               0
postal_code        11
region              0
product_id          0
category            0
sub-category        0
product_name        0
sales               0
dtype: int64

In [23]:
# Drop rows where order_date or ship_date is missing
df = df.dropna(subset=["order_date", "ship_date"])

print("Shape after dropping missing dates:", df.shape)
print("\nRemaining missing values:")
print(df.isnull().sum())

Shape after dropping missing dates: (2676, 18)

Remaining missing values:
row_id           0
order_id         0
order_date       0
ship_date        0
ship_mode        0
customer_id      0
customer_name    0
segment          0
country          0
city             0
state            0
postal_code      4
region           0
product_id       0
category         0
sub-category     0
product_name     0
sales            0
dtype: int64


In [24]:
# Fill missing postal_code with median
median_postal = df["postal_code"].median()
df["postal_code"] = df["postal_code"].fillna(median_postal)

print("Final missing values:")
print(df.isnull().sum())

Final missing values:
row_id           0
order_id         0
order_date       0
ship_date        0
ship_mode        0
customer_id      0
customer_name    0
segment          0
country          0
city             0
state            0
postal_code      0
region           0
product_id       0
category         0
sub-category     0
product_name     0
sales            0
dtype: int64


In [25]:
# -------- Cleaning Functions -------- #

def standardize_columns(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
    return df

def convert_dates(df):
    for col in df.columns:
        if "date" in col:
            df[col] = pd.to_datetime(df[col], errors="coerce")
    return df

def remove_duplicates(df):
    df = df.drop_duplicates()
    return df

def handle_missing(df):
    # Drop rows with missing date columns
    date_cols = [col for col in df.columns if "date" in col]
    df = df.dropna(subset=date_cols)

    # Fill numeric columns with median
    numeric_cols = df.select_dtypes(include=np.number).columns
    for col in numeric_cols:
        df[col] = df[col].fillna(df[col].median())

    # Fill categorical columns with mode
    categorical_cols = df.select_dtypes(include="object").columns
    for col in categorical_cols:
        df[col] = df[col].fillna(df[col].mode()[0])

    return df

def clean_data(df):
    df = standardize_columns(df)
    df = convert_dates(df)
    df = remove_duplicates(df)
    df = handle_missing(df)
    return df

In [26]:
# Reload raw data
raw_df = pd.read_csv("data/raw_data.csv")

# Apply cleaning pipeline
cleaned_df = clean_data(raw_df)

print("Shape after cleaning:", cleaned_df.shape)
print("\nMissing values after cleaning:")
print(cleaned_df.isnull().sum())

Shape after cleaning: (2676, 18)

Missing values after cleaning:
row_id           0
order_id         0
order_date       0
ship_date        0
ship_mode        0
customer_id      0
customer_name    0
segment          0
country          0
city             0
state            0
postal_code      0
region           0
product_id       0
category         0
sub-category     0
product_name     0
sales            0
dtype: int64


C:\Users\braje\AppData\Local\Temp\ipykernel_26552\3234895857.py:28: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  categorical_cols = df.select_dtypes(include="object").columns


In [27]:
# -------- Generate Cleaning Report -------- #

report = {}

report["original_shape"] = raw_df.shape
report["cleaned_shape"] = cleaned_df.shape
report["rows_removed"] = raw_df.shape[0] - cleaned_df.shape[0]
report["columns_count"] = cleaned_df.shape[1]
report["missing_values_after_cleaning"] = cleaned_df.isnull().sum().sum()

print("Cleaning Report:")
for key, value in report.items():
    print(f"{key}: {value}")

Cleaning Report:
original_shape: (9800, 18)
cleaned_shape: (2676, 18)
rows_removed: 7124
columns_count: 18
missing_values_after_cleaning: 0


In [28]:
# Save cleaned dataset
cleaned_df.to_csv("cleaned_output/cleaned_data.csv", index=False)

# Save cleaning report
with open("cleaned_output/cleaning_report.txt", "w") as f:
    for key, value in report.items():
        f.write(f"{key}: {value}\n")

print("Files saved successfully.")

Files saved successfully.


# Conclusion

A modular cleaning pipeline was developed to automate preprocessing tasks.  
The cleaned dataset contains no missing values and is ready for analysis or machine learning workflows.